<a href="https://colab.research.google.com/github/stranger9977/NFL-Machine-Learning/blob/master/Playoffs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [353]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('ETR FFPC Projections').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
ECRdf = pd.DataFrame.from_records(rows)

[['Player', 'Team', 'Position', 'Expected Games', 'Predicted Games', 'Ceiling Games', 'Predicted Points', 'X_FFPC Points', 'FFPC Points', 'Ceiling FFPC Points', 'Pos Rank', 'PPG', 'PPG Pos Rank'], ['Patrick Mahomes', 'KC', 'QB', '3.1', '4', '4', '103.6', '80.9', '129.5', '129.5', 'QB01', '25.9', 'QB02'], ['Josh Allen', 'BUF', 'QB', '2.4', '3', '4', '84.6', '68.7', '84.6', '141', 'QB02', '28.2', 'QB01'], ['Tom Brady', 'TB', 'QB', '2.8', '3', '4', '74.4', '68.3', '74.4', '124', 'QB03', '24.8', 'QB03'], ['Travis Kelce', 'KC', 'TE', '3.1', '4', '4', '78.4', '61.2', '98', '98', 'TE01', '19.6', 'TE01'], ['Cooper Kupp', 'LA', 'WR', '2.4', '2', '4', '47.6', '57.5', '47.6', '119', 'WR02', '23.8', 'WR01'], ['Tyreek Hill', 'KC', 'WR', '3.1', '4', '4', '74.8', '58.5', '93.5', '93.5', 'WR01', '18.7', 'WR04'], ['Aaron Rodgers', 'GB', 'QB', '2.3', '3', '3', '69.9', '52.5', '93.2', '', 'QB07', '23.3', 'QB06'], ['Matthew Stafford', 'LA', 'QB', '2.4', '2', '4', '45.2', '54.6', '45.2', '113', 'QB04', '22

In [354]:
new_header = ECRdf.iloc[0] #grab the first row for the header
ECRdf = ECRdf[1:] #take the data less the header row
ECRdf.columns = new_header #set the header row as the df header
ECRdf.head(100)

,Player,Team,Position,Expected Games,Predicted Games,Ceiling Games,Predicted Points,X_FFPC Points,FFPC Points,Ceiling FFPC Points,Pos Rank,PPG,PPG Pos Rank
1,Patrick Mahomes,KC,QB,3.1,4,4,103.6,80.9,129.5,129.5,QB01,25.9,QB02
2,Josh Allen,BUF,QB,2.4,3,4,84.6,68.7,84.6,141,QB02,28.2,QB01
3,Tom Brady,TB,QB,2.8,3,4,74.4,68.3,74.4,124,QB03,24.8,QB03
4,Travis Kelce,KC,TE,3.1,4,4,78.4,61.2,98,98,TE01,19.6,TE01
5,Cooper Kupp,LA,WR,2.4,2,4,47.6,57.5,47.6,119,WR02,23.8,WR01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Zay Jones,LV,WR,1.5,1,4,10.3,15.4,10.3,51.5,WR31,10.3,WR22
97,Cyril Grayson,TB,WR,2.8,3,4,15.3,14.2,15.3,25.5,WR34,5.1,WR44
98,Anthony Firkser,TEN,TE,2,2,3,13.4,13.6,13.4,,TE13,6.7,TE13
99,A.J. Green,ARI,WR,1.7,1,4,9.8,16.3,9.8,49,WR30,9.8,WR24


In [355]:
ECRdf.columns

Index(['Player', 'Team', 'Position', 'Expected Games', 'Predicted Games',
       'Ceiling Games', 'Predicted Points', 'X_FFPC Points', 'FFPC Points',
       'Ceiling FFPC Points', 'Pos Rank', 'PPG', 'PPG Pos Rank'],
      dtype='object', name=0)

In [356]:
ECRdf[['Expected Games','FFPC Points','PPG']] = ECRdf[['Expected Games','FFPC Points','PPG']].astype(float, errors = 'raise')


In [357]:
adp_df_cutoff = ECRdf[:101]

In [358]:
adp_df_cutoff.shape


(101, 13)

In [359]:
replacement_players = {
    'RB': '',
    'QB': '',
    'WR': '',
    'TE': '',
    'K': '',
    'DST': '',
}

In [360]:
adp_df_cutoff.columns

Index(['Player', 'Team', 'Position', 'Expected Games', 'Predicted Games',
       'Ceiling Games', 'Predicted Points', 'X_FFPC Points', 'FFPC Points',
       'Ceiling FFPC Points', 'Pos Rank', 'PPG', 'PPG Pos Rank'],
      dtype='object', name=0)

In [361]:
for _, row in adp_df_cutoff.iterrows():
    
    team = row['Team']
    position = row['Position'] # extract out the position and player value from each row as we loop through it
    player = row['Player']
    
    if position in replacement_players: # if the position is in the dict's keys
        replacement_players[position] = player # set that player as the replacement player

In [362]:
replacement_players

{'DST': 'DAL DST',
 'K': 'Randy Bullock',
 'QB': 'Mac Jones',
 'RB': 'James Conner',
 'TE': 'Geoff Swaim',
 'WR': 'A.J. Green'}

In [363]:
replacement_players_team = {'ARI': 'James Conner',
 'BUF': 'Gabriel Davis',
 'CIN': 'Tyler Boyd',
 'DAL': 'Dalton Schultz',
 'GB': 'AJ Dillon',
 'KC': 'Darrel Williams',
 'LA': 'Van Jefferson',
 'NE': 'Hunter Henry',
 'LV' : 'Hunter Renfrow',
 'PHI': 'Devonta Smith',
 'PIT': 'Diontae Johnson',
 'LA': 'Odell Beckham',
 'SF': 'Brandon Aiyuk',
 'TB': 'Mike Evans',
 'TEN': 'Julio Jones'}

In [364]:
replacement_players_team_qb = {'ARI': 'Zach Ertz',
 'BUF': 'Stefon Diggs',
 'CIN': "Ja'Marr Chase",
 'DAL': 'CeeDee Lamb',
 'GB': 'Davante Adams',
 'KC': 'Travis Kelce',
 'LA': 'Cooper Kupp',
 'NE': 'Jakobi Meyers',
 'LV' : 'Darren Waller',
 'PHI': 'Dallas Goedert',
 'PIT': 'Najee Harris',
 'SF': 'Deebo Samuel',
 'TB': 'Leonard Fournette',
 'TEN': 'A.J. Brown'}

In [365]:
replacement_players_team

{'ARI': 'James Conner',
 'BUF': 'Gabriel Davis',
 'CIN': 'Tyler Boyd',
 'DAL': 'Dalton Schultz',
 'GB': 'AJ Dillon',
 'KC': 'Darrel Williams',
 'LA': 'Odell Beckham',
 'LV': 'Hunter Renfrow',
 'NE': 'Hunter Henry',
 'PHI': 'Devonta Smith',
 'PIT': 'Diontae Johnson',
 'SF': 'Brandon Aiyuk',
 'TB': 'Mike Evans',
 'TEN': 'Julio Jones'}

In [366]:
replacement_players

{'DST': 'DAL DST',
 'K': 'Randy Bullock',
 'QB': 'Mac Jones',
 'RB': 'James Conner',
 'TE': 'Geoff Swaim',
 'WR': 'A.J. Green'}

In [367]:
ECRdf = ECRdf[['Player', 'Position', 'Team', 'FFPC Points']] # filtering out the columns we need.
ECRdf.head()

,Player,Position,Team,FFPC Points
1,Patrick Mahomes,QB,KC,129.5
2,Josh Allen,QB,BUF,84.6
3,Tom Brady,QB,TB,74.4
4,Travis Kelce,TE,KC,98.0
5,Cooper Kupp,WR,LA,47.6


In [368]:
replacement_values_team = {}

for team, player_name in replacement_players_team.items():
    player = ECRdf.loc[ECRdf['Player'] == player_name]

    replacement_values_team[team] = player['FFPC Points'].tolist()[0]

In [369]:
replacement_players_team_qb

{'ARI': 'Zach Ertz',
 'BUF': 'Stefon Diggs',
 'CIN': "Ja'Marr Chase",
 'DAL': 'CeeDee Lamb',
 'GB': 'Davante Adams',
 'KC': 'Travis Kelce',
 'LA': 'Cooper Kupp',
 'LV': 'Darren Waller',
 'NE': 'Jakobi Meyers',
 'PHI': 'Dallas Goedert',
 'PIT': 'Najee Harris',
 'SF': 'Deebo Samuel',
 'TB': 'Leonard Fournette',
 'TEN': 'A.J. Brown'}

In [370]:
replacement_values_team_qb = {}

for team, player_name in replacement_players_team_qb.items():
    player = ECRdf.loc[ECRdf['Player'] == player_name]

    replacement_values_team_qb[team] = player['FFPC Points'].tolist()[0]

In [371]:
replacement_values_team_qb

{'ARI': 16.7,
 'BUF': 53.7,
 'CIN': 32.4,
 'DAL': 32.2,
 'GB': 88.4,
 'KC': 98.0,
 'LA': 47.6,
 'LV': 18.0,
 'NE': 11.9,
 'PHI': 16.1,
 'PIT': 16.8,
 'SF': 19.0,
 'TB': 61.2,
 'TEN': 32.4}

In [372]:
replacement_values = {} # initialize an empty dictionary

for position, player_name in replacement_players.items():
    
    player = ECRdf.loc[ECRdf['Player'] == player_name]
    
    # because this is a series object we get back, we need to use the tolist method
    # to get back the series as a list. The list object is of length 1, and the 1 item has the value we need.
    # we tack on a [0] to get the value we need.
    
    replacement_values[position] = player['FFPC Points'].tolist()[0]

In [373]:
replacement_values

{'DST': 12.2, 'K': 16.2, 'QB': 15.4, 'RB': 11.0, 'TE': 12.4, 'WR': 9.8}

In [374]:
pd.set_option('chained_assignment', None)

ECRdf = ECRdf.loc[ECRdf['Position'].isin(['QB','RB', 'WR', 'TE','K',"DST"])]

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html
ECRdf['VOR'] = ECRdf.apply(
    lambda row: row['FFPC Points'] - replacement_values.get(row['Position']), axis=1
)

In [375]:
ECRdf = ECRdf.loc[ECRdf['Position'].isin(['QB','RB', 'WR', 'TE','K',"DST"])]

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html
ECRdf['VOR_Team_QB'] = ECRdf.apply(
    lambda row: row['FFPC Points'] - replacement_values_team_qb.get(row['Team']), axis=1
)

In [376]:
pd.set_option('chained_assignment', None)

ECRdf = ECRdf.loc[ECRdf['Team'].isin(['ARI',
 'BUF',
 'CIN',
 'DAL',
 'GB',
 'KC',
 'LA',
 'LV',
 'NE',
 'PHI',
 'PIT',
 'SF',
 'TB',
 'TEN'])]

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html
ECRdf['VOR_Team'] = ECRdf.apply(
    lambda row: row['FFPC Points'] - replacement_values_team.get(row['Team']), axis=1
)

In [377]:
ECRdf.head(101)

,Player,Position,Team,FFPC Points,VOR,VOR_Team_QB,VOR_Team
1,Patrick Mahomes,QB,KC,129.5,114.1,31.5,65.0
2,Josh Allen,QB,BUF,84.6,69.2,30.9,55.5
3,Tom Brady,QB,TB,74.4,59.0,13.2,28.2
4,Travis Kelce,TE,KC,98.0,85.6,0.0,33.5
5,Cooper Kupp,WR,LA,47.6,37.8,0.0,23.4
...,...,...,...,...,...,...,...
97,Cyril Grayson,WR,TB,15.3,5.5,-45.9,-30.9
98,Anthony Firkser,TE,TEN,13.4,1.0,-19.0,-3.6
99,A.J. Green,WR,ARI,9.8,0.0,-6.9,-1.2
100,Geoff Swaim,TE,TEN,12.4,0.0,-20.0,-4.6


In [378]:
ECRdf["Composite"] = ECRdf['VOR']+ ECRdf['VOR_Team']

In [379]:
ECRdf.head(100)

,Player,Position,Team,FFPC Points,VOR,VOR_Team_QB,VOR_Team,Composite
1,Patrick Mahomes,QB,KC,129.5,114.1,31.5,65.0,179.1
2,Josh Allen,QB,BUF,84.6,69.2,30.9,55.5,124.7
3,Tom Brady,QB,TB,74.4,59.0,13.2,28.2,87.2
4,Travis Kelce,TE,KC,98.0,85.6,0.0,33.5,119.1
5,Cooper Kupp,WR,LA,47.6,37.8,0.0,23.4,61.2
...,...,...,...,...,...,...,...,...
96,Zay Jones,WR,LV,10.3,0.5,-7.7,-1.8,-1.3
97,Cyril Grayson,WR,TB,15.3,5.5,-45.9,-30.9,-25.4
98,Anthony Firkser,TE,TEN,13.4,1.0,-19.0,-3.6,-2.6
99,A.J. Green,WR,ARI,9.8,0.0,-6.9,-1.2,-1.2


In [380]:
gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('FFPC Ownership').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
df = pd.DataFrame.from_records(rows)

[['Player', 'Position', 'Team', 'Projected Own', 'Projected Own fixed'], ['Davante Adams', 'WR', 'GB', '78%', '78'], ['Cooper Kupp', 'WR', 'LA', '76%', '76'], ['No PHI', 'No Player', 'PHI', '67%', '67'], ['Derrick Henry', 'RB', 'TEN', '60%', '60'], ['Travis Kelce', 'TE', 'KC', '49%', '49'], ['Leonard Fournette', 'RB', 'TB', '43%', '43'], ['No PIT', 'No Player', 'PIT', '43%', '43'], ['Stefon Diggs', 'WR', 'BUF', '38%', '38'], ['NE DST', 'DEF', 'NE', '37%', '37'], ['Joe Mixon', 'RB', 'CIN', '35%', '35'], ['Deebo Samuel', 'WR', 'SF', '34.00%', '34'], ['Josh Allen', 'QB', 'BUF', '30%', '30'], ['ARZ DST', 'DEF', 'ARI', '28%', '28'], ['Ezekiel Elliott', 'RB', 'DAL', '28.00%', '28'], ["Ja'Marr Chase", 'WR', 'CIN', '28%', '28'], ['George Kittle', 'TE', 'SF', '28.00%', '28'], ['Patrick Mahomes', 'QB', 'KC', '28%', '28'], ['Daniel Carlson', 'K', 'LV', '28%', '28'], ['No LV', 'No Player', 'LV', '27%', '27'], ['A.J. Brown', 'WR', 'TEN', '27%', '27'], ['Najee Harris', 'RB', 'PIT', '26%', '26'], ['R

In [381]:
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df.head(100)

,Player,Position,Team,Projected Own,Projected Own fixed
1,Davante Adams,WR,GB,78%,78
2,Cooper Kupp,WR,LA,76%,76
3,No PHI,No Player,PHI,67%,67
4,Derrick Henry,RB,TEN,60%,60
5,Travis Kelce,TE,KC,49%,49
...,...,...,...,...,...
96,Tyler Higbee,TE,LA,1%,1
97,Tony Pollard,RB,DAL,1.00%,1
98,No KC,No Player,KC,1%,1
99,Cedrick Wilson,WR,DAL,1.00%,1


In [382]:
#Export to Google Sheets / Part 2 Export
from gspread_dataframe import get_as_dataframe, set_with_dataframe

sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1vn2_WbyDgy2gIo00fiOsPzDve09hE4_Qg97MhqYzfxg/edit#gid=1824609219')
worksheet = sh.get_worksheet(1)

set_with_dataframe(worksheet,ECRdf )